# Functional Responses

**How well do different mathematical models, e.g., based upon foraging theory (mechanistic) principles vs. phenomenological ones, fit to functional responses data across species?**

Functional responses:
  * Are the relationships between a consumer's biomass consumption rate and abundance of a target resource. 

  * Arise from fundamental biological and physical constraints on consumer-resource interactions.

  * Determine the rate of biomass flow between species in ecosystems across the full scale of sizes. 

  * Play a key role in determining the stability of food webs that underpin ecosystems

NB: Two main fields of interest are **N_TraitValue** (the number of resources consumed per consumer per unit time) and **ResDensity** (the resource abundance). 

Each ID value corresponds to one curve and one individual functional response curve

In [1]:
# Some imports to explore the datasets
import pandas as pd
import scipy as sc
import matplotlib.pylab as pl
import seaborn as sns
import numpy as np
import csv

## Getting Started

In [2]:
data = pd.read_csv("../data/CRat.csv")
print("Loaded {} columns.".format(len(data.columns.values)))

Loaded 68 columns.


In [3]:
print(data.columns.values)

['ID' 'DataType' 'ORIGINAL_TraitID' 'ORIGINAL_TraitDefinition' 'TraitValue'
 'TraitUnit' 'N_TraitValue' 'ConTaxa' 'ResTaxa' 'ConTaxaStage'
 'ResTaxaStage' 'Con_ForagingMovement' 'Con_RESDetectionDimensionality'
 'Res_ForagingMovement' 'Res_CONDetectionDimensionality' 'CON_MASS_value'
 'RES_MASS_value' 'ResArenaSize_SI_UNIT' 'ResDensity_SI_VALUE'
 'ResDensityUnit' 'ResDensity' 'ResArenaSize_SI_VALUE' 'ConTemp' 'ResTemp'
 'ResReplaceRate' 'ResReplaceUnit' 'ResReplace' 'TraitSIValue'
 'TraitSIUnit' 'N_TraitID' 'N_TraitConversion' 'N_CONVERTED' 'N_TraitUnit'
 'Original_ErrorValue' 'Original_ErrorValueUnit' 'Replicates' 'ConTaxon'
 'ConStage' 'ConCommon' 'Con_MovementDimensionality' 'Con_Thermy'
 'Res_MovementDimensionality' 'ResTaxon' 'ResStage' 'ResCommon'
 'Res_Thermy' 'Habitat' 'LabField' 'ObservationtimeSI'
 'ObservationtimeSIUnits' 'ConStarvationTimeSI' 'ConStarvationTimeSIUnits'
 'ConArenaSiz_VALUE' 'ConArenaSize_UNIT' 'ConDensity' 'ConDensityUnit'
 'ConDensity_SI_VALUE' 'ConDensityC

#### Making new data frame with only columns of interest for initial plots

In [4]:
newdata = data[['ID','ResDensity','N_TraitValue']].copy()
print(newdata)

         ID  ResDensity  N_TraitValue
0     39835     10.0000      0.001206
1     39835     10.0000      0.001065
2     39835     10.0000      0.001206
3     39835     25.0000      0.002562
4     39835     25.0000      0.002411
5     39835     25.0000      0.002009
6     39835     25.0000      0.001587
7     39835     50.0000      0.004330
8     39836     19.9323      0.000525
9     39836     40.1737      0.000950
10    39836     60.0883      0.001321
11    39836     80.3182      0.001647
12    39836    100.2190      0.001900
13    39836    120.1230      0.002181
14    39836    140.3380      0.002371
15    39836    160.2320      0.002570
16    39836    180.4470      0.002760
17    39836    200.3350      0.002904
18    39837     19.9323      0.000255
19    39837     40.1737      0.000483
20    39837     60.0883      0.000702
21    39837     80.3182      0.000894
22    39837    100.2190      0.001076
23    39837    120.1230      0.001241
24    39837    140.3380      0.001387
25    39837 

Check if there are any columns with NA values

In [5]:
newdata2 = newdata[newdata.isna().any(axis=1)]
print(newdata2)

Empty DataFrame
Columns: [ID, ResDensity, N_TraitValue]
Index: []


#### Removing IDs with less than 5 data points

In [0]:
newdata.groupby('ID').ID.count() # grouping data points by ID number and storing a count of instances

Setting a threshold number of repeats. If a ID has less than the threshold number of repeats, all of its data is replaced with null values

In [0]:
threshold = 5
ValueCounts = newdata['ID'].value_counts()
toremove = ValueCounts[ValueCounts <= threshold].index
newdata.replace(toremove, np.nan, inplace=True)

In [0]:
newdata['ID'].value_counts() #Counting number of repeated IDs 

Remove all rows with NA data

In [0]:
newdata_no_missing = newdata.dropna()

In [0]:
newdata_no_missing.isnull().sum() #count number of NAs in data set to be sure

#### Save modified data into a .csv file

In [0]:
newdata_no_missing.to_csv('../data/FunResData.csv')

## NLLS Fitting Script

In [0]:
print(newdata)